In [1]:
pip install pymongo opencv-python face_recognition numpy

In [2]:
import cv2
import face_recognition
from pymongo import MongoClient
import numpy as np
import tkinter as tk
from tkinter import messagebox

class BiometricSimulator:
    def __init__(self, db_name='biometric_db'):
        self.client = MongoClient('mongodb://localhost:27017/')
        self.db = self.client[db_name]
        self.collection = self.db['users']

    def is_username_taken(self, username):
        """Verifica si el nombre de usuario ya está registrado."""
        return self.collection.find_one({'username': username}) is not None

    def register_user(self, username, password):
        """Registra un nuevo usuario."""
        if self.is_username_taken(username):
            messagebox.showerror("Error", "El nombre de usuario ya está en uso. Por favor, elige otro.")
            return

        user_data = {
            'username': username,
            'password': password,
            'face_data': []  # Almacena datos de rostro
        }
        self.collection.insert_one(user_data)
        messagebox.showinfo("Registro", f"Usuario {username} registrado con éxito.")

    def capture_face(self, username):
        """Captura y almacena la imagen facial del usuario."""
        cap = cv2.VideoCapture(0)
        face_data = []

        # Crear una ventana para mostrar la captura
        capture_window = tk.Toplevel()
        capture_window.title("Captura de Rostro")
        capture_window.geometry("640x480")

        def capture_image():
            ret, frame = cap.read()
            if ret:
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                face_data.append(rgb_frame)  # Guarda la imagen en formato RGB
                cv2.imshow('Captura de Rostro', frame)
                messagebox.showinfo("Captura", "Imagen facial capturada. Presiona 'Guardar' para almacenar.")

        def save_face():
            if face_data:
                # Convertir cada imagen a formato RGB antes de almacenarla
                rgb_face_data = [cv2.cvtColor(img, cv2.COLOR_RGB2BGR) for img in face_data]
                self.collection.update_one({'username': username}, {'$set': {'face_data': [img.tolist() for img in rgb_face_data]}})
                messagebox.showinfo("Almacenamiento", f"Datos faciales del usuario {username} guardados con éxito.")
                capture_window.destroy()
            else:
                messagebox.showerror("Error", "No se ha capturado ninguna imagen facial.")

        # Botones de captura
        capture_button = tk.Button(capture_window, text="Capturar Imagen", command=capture_image, bg='#28a745', fg='white', font=('Helvetica', 14))
        capture_button.pack(pady=10)

        save_button = tk.Button(capture_window, text="Guardar", command=save_face, bg='#007BFF', fg='white', font=('Helvetica', 14))
        save_button.pack(pady=10)

        # Cerrar la cámara y la ventana al cerrar la ventana de captura
        def on_closing():
            cap.release()
            cv2.destroyAllWindows()
            capture_window.destroy()

        capture_window.protocol("WM_DELETE_WINDOW", on_closing)

    def verify_user(self, username, password):
        """Verifica el usuario y la contraseña."""
        user_data = self.collection.find_one({'username': username})
        if user_data and user_data['password'] == password:
            return user_data
        return None

    def verify_face(self, username):
        """Verifica la identificación facial del usuario a partir de la imagen capturada."""
        user_data = self.collection.find_one({'username': username})
        stored_face_data = user_data['face_data']

        if not stored_face_data:
            messagebox.showerror("Error", "No hay datos faciales almacenados para este usuario.")
            return False

        # Convertir la imagen almacenada de lista a matriz NumPy
        stored_face_data = np.array(stored_face_data[0], dtype=np.uint8)
        stored_face_encodings = face_recognition.face_encodings(stored_face_data)

        if not stored_face_encodings:
            messagebox.showerror("Error", "No se pudo codificar el rostro almacenado.")
            return False

        # Captura de la imagen del usuario para verificación
        cap = cv2.VideoCapture(0)
        ret, frame = cap.read()
        cap.release()

        if not ret:
            messagebox.showerror("Error", "No se pudo capturar la imagen.")
            return False

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        captured_face_encodings = face_recognition.face_encodings(rgb_frame)

        if not captured_face_encodings:
            messagebox.showerror("Error", "No se detectó ningún rostro en la imagen capturada.")
            return False

        # Comparar la codificación capturada con la almacenada
        match = face_recognition.compare_faces([stored_face_encodings[0]], captured_face_encodings[0])

        if match[0]:
            messagebox.showinfo("Verificación", f"¡Bienvenido, {username}!")
            return True
        else:
            messagebox.showerror("Verificación", "Verificación fallida. Rostro no reconocido.")
            return False

    def run(self):
        """Ejecuta la interfaz gráfica."""
        root = tk.Tk()
        root.title("Simulador Biométrico")
        root.geometry("400x300")

        tk.Label(root, text="Nombre de Usuario").pack(pady=10)
        username_entry = tk.Entry(root)
        username_entry.pack(pady=5)

        tk.Label(root, text="Contraseña").pack(pady=10)
        password_entry = tk.Entry(root, show="*")
        password_entry.pack(pady=5)

        button_style = {'bg': '#007BFF', 'fg': 'white', 'font': ('Helvetica', 14)}

        def login():
            username = username_entry.get()
            password = password_entry.get()
            if self.verify_user(username, password):
                self.verify_face(username)

        def register():
            username = username_entry.get()
            password = password_entry.get()
            self.register_user(username, password)
            self.capture_face(username)

        tk.Button(root, text="Iniciar Sesión", command=login, **button_style).pack(pady=10)
        tk.Button(root, text="Registrarse", command=register, **button_style).pack(pady=10)

        # Ejecutar la interfaz gráfica
        root.mainloop()

if __name__ == "__main__":
    simulator = BiometricSimulator()
    simulator.run()
